In [1]:
# import preprocess as prep
import SLR_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# from preprocess import person

# model will output multiple(15) results/sec, how are we gonna handle it?

2024-10-16 17:18:40.735866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 17:18:40.743589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 17:18:40.745794: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 17:18:40.751306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1729066721.508364 12179

In [2]:
importlib.reload(SLR_model)
importlib.reload(preprocess)


<module 'preprocess' from '/home/shd/BomNae-SLR/preprocess.py'>

In [3]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16

In [4]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "check_16-15-00_1_3time.keras")
model = SLR_model.load_model(load_path)

In [5]:
models=[]
for i in range(1,15):
    load_path=os.path.join(model_dir, "check_16-15-07_"+str(i)+"_100epochs_1time.keras")
    models.append(SLR_model.load_model(load_path))

In [6]:
print(models)

[<SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>, <SLRModel name=slr_model_4, built=True>]


In [7]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 1
# TODO hardcode path
# a video
# for j in os.listdir(os.path.join(preprocess.getoutputdir("val"),str(person_id))):
#     j="0005.npz"
#     l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"{person_id}/{j}","val")
    
for j in sorted(os.listdir(preprocess.getoutputdir:
    l_raw, r_raw, p_raw, y_raw,w = preprocess.load_data(f"{person_id}/0005.npz",)
    
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    break

In [8]:
y_raws

[5]

In [9]:
# # dummy x_raw data
# l_raws = []
# duration_list = []
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = (np.random.randint(90,100))
# duration_list.append(duration)
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# l_raws.append(x_elem)
# r_raws = []
# 
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# r_raws.append(x_elem)
# p_raws = []
# 
# # h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     # x_elem.append(h_elem)
#     x_elem.append(p_elem)
#     # x_elem.append(comb_elem)
# p_raws.append(x_elem)

In [10]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [11]:
x_test[1].shape

(9, 63, 4, 5, 3)

In [12]:
for model in models:
    predictions = model.predict(x_test,20000)
    print(SLR_model.decode_onehot2d(predictions))


I0000 00:00:1729066728.423253 1218075 service.cc:146] XLA service 0x7ff4080031b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729066728.423304 1218075 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-10-16 17:18:48.437968: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-16 17:18:48.500819: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 886ms/step


I0000 00:00:1729066729.156451 1218075 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


tf.Tensor([2533 2379 2427 2427 2348 2533 2515 2517 2517], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
tf.Tensor([2349 2466 2348 2466 2349 2430 2349 2466 2398], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
tf.Tensor([2032 2032 2142 2056  385 2055 2053 2032 2142], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
tf.Tensor([650 650 650 650 650 650 650 650 619], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step
tf.Tensor([836 780 780 780 780 780 780 780 836], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
tf.Tensor([2771 2744 2636 2640 2640 1691 2640 1691  237], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
tf.Tensor([787 798 971 798 798 159 798 798 798], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
tf.Tensor([619 619 619 643 643 720 650 650 650], shape=(9,), dtype=int64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
tf.Tensor([ 499  499 1284 1284 1284 1284 1284 1284  499], shape

In [13]:
predictions

array([[7.9010423e-16, 3.3546442e-19, 2.8880013e-28, ..., 6.2367728e-19,
        4.2299254e-18, 1.5826244e-16],
       [1.3019671e-17, 2.2345743e-21, 2.4805354e-28, ..., 7.5384776e-20,
        2.5882523e-18, 5.4566355e-18],
       [1.9817608e-20, 1.7838993e-23, 4.4732374e-32, ..., 1.6420462e-22,
        1.9081687e-20, 6.3201866e-21],
       ...,
       [4.3076135e-23, 1.4819824e-27, 1.0392310e-33, ..., 3.6207864e-27,
        5.4690183e-25, 3.4301395e-23],
       [4.0488436e-22, 5.7491828e-25, 7.4267833e-32, ..., 1.9858946e-25,
        2.9434539e-22, 5.8357614e-23],
       [3.3527245e-23, 1.4266392e-26, 9.7643908e-32, ..., 3.7197943e-27,
        7.2161421e-25, 5.1433125e-25]], dtype=float32)

In [14]:
# pred_bin = predictions.round().astype(np.int32)
# pred_bin

In [15]:
importlib.reload(SLR_model)

<module 'SLR_model' from '/home/shd/BomNae-SLR/SLR_model.py'>

In [16]:
SLR_model.decode_onehot2d(predictions)

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([1935, 1935, 1891, 1891, 1891, 1935, 1891, 1891, 1935])>

In [17]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

AttributeError: module 'SLR_model' has no attribute 'bin2num'

In [ ]:
model.predict